# 도서 정보 객체

In [16]:
class Book:
    def __init__(self, rank, title, author, price):
        self.rank = rank
        self.title = title
        self.author = author
        self.price = price

    def __str__(self):
        return f"{self.rank}, {self.title}, {self.author}, {self.price}"
    
    def to_dict(self):
        return {'rank':self.rank, 
                'title':self.title, 
                'author':self.author, 
                'price':self.price}
    
    def to_list(self):
        return [self.rank, 
                self.title, 
                self.author, 
                self.price]

#yesBestList > li:nth-child(1) > div > div.item_info

In [17]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
best_list_el = soup.select('#yesBestList div.item_info')
len(best_list_el)


24

In [18]:
best_list_el[0]

<div class="item_info">
<div class="info_row info_keynote">
<span class="gd_keynote" id="spanGdKeynote">
<a class="iconC tBook" href="/24/Category/More/001014?ElemNo=13&amp;ElemSeq=1&amp;Gcode=000_030_001"><em class="txt">오늘의책</em></a>
<!-- 클래스24 상품일 경우 -->
</span>
<script type="text/javascript">
        if ($('#spanGdKeynote').children().length == 0) {
            $('#spanGdKeynote').remove();
        }
    </script>
</div>
<div class="info_row info_name">
<span class="gd_res">[도서]</span>
<a class="gd_name" href="/product/goods/143911524" onclick="wiseLogV2('BS', '001_005_001', ''); ">혼모노</a>
<a class="bgYUI ico_nWin" href="/product/goods/143911524" onclick="wiseLogV2('BS', '001_005_001', ''); " target="_blank">혼모노 새창이동</a>
</div>
<div class="info_row info_pubGrp">
<span class="authPub info_auth" onclick="wiseLogV2('BS', '001_005_002', '');">
<a href="https://www.yes24.com/product/search?domain=ALL&amp;query=%25EC%2584%25B1%25ED%2595%25B4%25EB%2582%2598&amp;authorNo=253719&amp;author=

In [19]:
book_list = []

for i, item in enumerate(best_list_el):
    title = item.select_one('div.info_name > a').text
    author = item.select_one('.info_auth > a').text
    price = item.select_one('div.info_price > strong').text
    book_list.append(Book(i+1, title, author, price))

for book in book_list:
    print(book)

1, 혼모노, 성해나, 16,200원
2, 류수영의 평생 레시피, 류수영, 22,500원
3, 가공범, 히가시노 게이고, 19,800원
4, 돌비공포라디오 더 레드, 돌비, 17,820원
5, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상, 최태성, 14,850원
6, 박곰희 연금 부자 수업, 박곰희, 18,900원
7, 2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급), 최태성, 17,550원
8, 단 한 줄만 내 마음에 새긴다고 해도, 나민애, 21,420원
9, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하, 최태성, 14,400원
10, ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC, ETS, 17,820원
11, ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC, ETS, 17,820원
12, 료의 생각 없는 생각, 료, 18,000원
13, 모순, 양귀자, 11,700원
14, 안녕이라 그랬어, 김애란, 15,120원
15, 자몽살구클럽, 한로로, 10,800원
16, 인생을 바꾸는 최고의 ETF, 잼투리, 22,050원
17, 청춘의 독서, 유시민, 17,010원
18, 소년이 온다, 한강, 13,500원
19, 첫 여름, 완주, 김금희, 15,300원
20, 경험의 멸종, 크리스틴 로젠, 17,820원
21, 어른의 행복은 조용하다, 태수, 16,020원
22, 견우와 선녀 대본집 세트, 양지훈, 41,400원
23, 어른의 품격을 채우는 100일 필사 노트, 김종원, 18,000원
24, 야구선수 김원중, 김원중, 18,000원


In [20]:
query = '''
CREATE TABLE IF NOT EXISTS BOOKS(
    RANK INTEGER PRIMARY KEY,
    TITLE TEXT,
    AUTHOR TEXT,
    PRICE INTEGER
)
'''

In [21]:
import sqlite3
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()
cursor.execute(query)
conn.commit()
conn.close()

In [22]:
ins_query = '''
INSERT INTO BOOKS (RANK, TITLE, AUTHOR, PRICE) VALUES (?,?,?,?)
'''

conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

for b in book_list:  
    cursor.execute(ins_query, b.to_list())

conn.commit()
conn.close()

# db table에서 읽어오기

In [23]:
def fetch_book_list_from_db():
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    
    try:
        cursor.execute('select * from books')
        rows = cursor.fetchall()
        return rows
    finally:
        conn.close()

# 여러페이지 가져오기

In [24]:
books = fetch_book_list_from_db()
for book in books:
    print(book)

(1, '혼모노', '성해나', '16,200원')
(2, '류수영의 평생 레시피', '류수영', '22,500원')
(3, '가공범', '히가시노 게이고', '19,800원')
(4, '돌비공포라디오 더 레드', '돌비', '17,820원')
(5, '2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상', '최태성', '14,850원')
(6, '박곰희 연금 부자 수업', '박곰희', '18,900원')
(7, '2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급)', '최태성', '17,550원')
(8, '단 한 줄만 내 마음에 새긴다고 해도', '나민애', '21,420원')
(9, '2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하', '최태성', '14,400원')
(10, 'ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC', 'ETS', '17,820원')
(11, 'ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC', 'ETS', '17,820원')
(12, '료의 생각 없는 생각', '료', '18,000원')
(13, '모순', '양귀자', '11,700원')
(14, '안녕이라 그랬어', '김애란', '15,120원')
(15, '자몽살구클럽', '한로로', '10,800원')
(16, '인생을 바꾸는 최고의 ETF', '잼투리', '22,050원')
(17, '청춘의 독서', '유시민', '17,010원')
(18, '소년이 온다', '한강', '13,500원')
(19, '첫 여름, 완주', '김금희', '15,300원')
(20, '경험의 멸종', '크리스틴 로젠', '17,820원')
(21, '어른의 행복은 조용하다', '태수', '16,020원')
(22, '견우와 선녀 대본집 세트', '양지훈', '41,400원')
(23, '어른의 품격을 채우는 100일 필사 노트', '김종원', '18,000원')
(24, '야구선수 김

In [25]:
base_url = 'https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageSize=24&pageNumber='
page_no = 3
book_list = []

# 페이지 연결
for page in range(1, page_no+1):
    url = f'{base_url}{page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    best_list_el = soup.select('#yesBestList div.item_info')

    # 요소 추출
    for i, item in enumerate(best_list_el):
        title = item.select_one('div.info_name > a').text
        author = item.select_one('.info_auth > a').text
        price = item.select_one('div.info_price > strong').text
        book_list.append(Book(i+1, title, author, price))
    
len(book_list)

72

In [26]:
for book in book_list:
    print(book)

1, 혼모노, 성해나, 16,200원
2, 류수영의 평생 레시피, 류수영, 22,500원
3, 가공범, 히가시노 게이고, 19,800원
4, 돌비공포라디오 더 레드, 돌비, 17,820원
5, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상, 최태성, 14,850원
6, 박곰희 연금 부자 수업, 박곰희, 18,900원
7, 2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급), 최태성, 17,550원
8, 단 한 줄만 내 마음에 새긴다고 해도, 나민애, 21,420원
9, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하, 최태성, 14,400원
10, ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC, ETS, 17,820원
11, ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC, ETS, 17,820원
12, 료의 생각 없는 생각, 료, 18,000원
13, 모순, 양귀자, 11,700원
14, 안녕이라 그랬어, 김애란, 15,120원
15, 자몽살구클럽, 한로로, 10,800원
16, 인생을 바꾸는 최고의 ETF, 잼투리, 22,050원
17, 청춘의 독서, 유시민, 17,010원
18, 소년이 온다, 한강, 13,500원
19, 첫 여름, 완주, 김금희, 15,300원
20, 경험의 멸종, 크리스틴 로젠, 17,820원
21, 어른의 행복은 조용하다, 태수, 16,020원
22, 견우와 선녀 대본집 세트, 양지훈, 41,400원
23, 어른의 품격을 채우는 100일 필사 노트, 김종원, 18,000원
24, 야구선수 김원중, 김원중, 18,000원
1, 다크 심리학, 다크 사이드 프로젝트, 19,710원
2, 궤도, 서맨사 하비, 15,300원
3, 먹어 보면 알지 : 호랑수박의 전설, 이지은, 15,300원
4, 해커스 토익 기출 VOCA (보카), David Cho, 11,610원
5, 2026 유휘운 행정법총론 요약노트+기출문제 (요.플.)

# mysql에 넣기

In [30]:
import pymysql

conn = pymysql.connect(
    host='localhost', user='root', password='1234',  db='my_db',
    charset='utf8'
) 

cursor = conn.cursor()

for book in book_list:
    price = int(book.price.replace(',', '').replace('원', '').strip())
    sql = "INSERT INTO books (title, author, price) VALUES (%s, %s, %s)"
    cursor.execute(sql, (book.title, book.author, price))

conn.commit()
cursor.close()
conn.close()
